In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
Cannot load cv2.


Setting environment for Precision WorkStation


No vtk
Cannot load cv2.


In [2]:
import logging

# This does not output anything on workstation, for some reason
# See https://stackoverflow.com/questions/18786912/get-output-from-the-logging-module-in-ipython-notebook/21475297#21475297
# head = '%(asctime)-15s %(message)s'
#logging.basicConfig(level=logging.DEBUG, format=head) 

# This works on workstation
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter( '%(asctime)-15s %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [3]:
classifier_id = 70
classifier_properties = classifier_settings.loc[classifier_id]

margin = classifier_properties['margin']
# model = classifier_properties['model']
sample_weighting = classifier_properties['sample_weighting']
neg_composition = classifier_properties['neg_composition']

In [4]:
model_dir_name = 'inception-bn-blue-softmax'
download_from_s3(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name), is_dir=True)
model_name = 'inception-bn-blue-softmax'
model_iteration = 0
output_symbol_name = 'softmax_output'
output_dim = 1024
mean_img = np.load(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, 'mean_224.npy'))

# Reference on how to predict with mxnet model:
# https://github.com/dmlc/mxnet-notebooks/blob/master/python/how_to/predict.ipynb
model_prefix = os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, model_name)
model0, arg_params, aux_params = mx.model.load_checkpoint(model_prefix, 0)

In [5]:
# Finetune
# http://mxnet.io/how_to/finetune.html

In [6]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten'):
    """
    symbol: the pretrained network symbol
    arg_params: the argument parameters of the pretrained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name + '_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = {k: arg_params[k] for k in arg_params if 'fc1' not in k}
    return (net, new_args)

In [7]:
num_classes = 2
(new_sym, new_args) = get_fine_tune_model(model0, arg_params, num_classes)

In [8]:
def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus, num_epoch, epoch_end_callback):
    devs = [mx.gpu(i) for i in range(num_gpus)]
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.fit(train, val,
        num_epoch=num_epoch,
        arg_params=arg_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback = mx.callback.Speedometer(batch_size, 10),
        kvstore='device',
        optimizer='sgd',
        optimizer_params={'learning_rate':0.01},
        initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        eval_metric='acc',
           epoch_end_callback=epoch_end_callback)
    metric = mx.metric.Accuracy()
    return mod.score(val, metric)

In [9]:
batch_per_gpu = 16
num_gpus = 1
batch_size = batch_per_gpu * num_gpus

In [10]:
from mxnet.model import save_checkpoint

def my_epoch_end_callback(prefix, period=1):
    def _callback(epoch, sym, arg, aux):    
        if epoch % period == 0:
            save_checkpoint(prefix, epoch, sym, arg, aux)
            symbol_fp = '%s-symbol.json' % prefix
            param_fp = '%s-%04d.params' % (prefix, epoch)
            upload_to_s3(symbol_fp)
            upload_to_s3(param_fp)
    return _callback

In [ ]:
# for structure in structures_found:
# for structure in all_known_structures:
for structure in ['LRt']:

    print structure

    # Determine which labels to load.

    structures_to_sample = [structure]
    negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

    margins_to_sample = [margin] # (200: 100 um, 500: 250 um)
    surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                                 for m in margins_to_sample
                                 for s in structures_to_sample 
                                 for surr_l in all_known_structures
                                 if surr_l != s]
    surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                                 for m in margins_to_sample
                                 for s in structures_to_sample]

    if neg_composition == 'neg_has_everything_else':
        labels_to_sample = structures_to_sample + negative_labels_to_sample
    elif neg_composition == 'neg_has_only_surround_noclass':
        labels_to_sample = structures_to_sample + surround_noclass_labels_to_sample + ['noclass']
    elif neg_composition == 'neg_has_all_surround':
        labels_to_sample = structures_to_sample + surround_positive_labels_to_sample + surround_noclass_labels_to_sample + ['noclass']
    
    # labels_to_sample = ['Sp5C', 'Sp5C_surround_500_Sp5I', 'Sp5C_surround_500_noclass', 'Sp5C_surround_500_LRt']
    # labels_to_sample = ['Sp5O', 'Sp5O_surround_500_Sp5I', 'Sp5O_surround_500_noclass']
#     labels_to_sample = ['SC', 'SC_surround_500_IC', 'SC_surround_500_noclass']

    # Load training dataset.

    training_set_ids = map(int, str(classifier_properties['train_set_id']).split('/'))
    training_features, training_addresses = load_dataset_images(training_set_ids, labels_to_sample=labels_to_sample)

    # convert patches data shape to nx1x224x224
    training_features = {s: (patches - mean_img)[:, None, :, :] for s, patches in training_features.iteritems()}

    # check which labels are collected
    labels_found = training_addresses.keys()
    structures_found = set([convert_to_original_name(l) for l in labels_found 
                            if convert_to_original_name(l) in labels_found]) - {'noclass'}

    # Load test dataset.

    test_set_ids = [62]
    test_features, test_addresses = load_dataset_images(test_set_ids, labels_to_sample=labels_to_sample)
    test_features = {s: (patches - mean_img)[:, None, :, :] for s, patches in test_features.iteritems()}
    
    #############################################
    
    
    if neg_composition == 'neg_has_only_surround_noclass':
        neg_classes = [convert_to_surround_name(structure, margin=margin, suffix='noclass')]
    elif neg_composition == 'neg_has_all_surround':
        neg_classes = [convert_to_surround_name(structure, margin=margin, suffix='noclass')]
        for surr_s in structures_found:
            c = convert_to_surround_name(structure, margin=margin, suffix=surr_s)
            if c in labels_found:
                neg_classes.append(c)
    elif neg_composition == 'neg_has_everything_else':
        neg_classes = [structure + '_negative']
    else:
        raise Exception('neg_composition %s is not recognized.' % neg_composition)

    ###########################
    ## Define Sample Weights ##
    ###########################
    
#     if sample_weighting == 'weighted':
#         neg_distances = np.concatenate([distances_to_structures[neg_class][structure] for neg_class in neg_classes])
            
#         sample_weights_neg = np.ones((n_neg,))
#         sample_weights_neg[neg_distances > thresh] = diminishing(neg_distances[neg_distances > thresh])
#         sample_weights = np.r_[np.ones((n_pos,)), sample_weights_neg]
#     else:
#         sample_weights = None
    
    ###########################################################################################
    
    train_features_pos = training_features[structure]
    n_pos = len(train_features_pos)
    
    train_features_neg = np.concatenate([training_features[neg_class] for neg_class in neg_classes])
    n_neg = len(train_features_neg)
    
    train_data = np.concatenate([train_features_pos, train_features_neg])
    # For cnn, labels must be 0/1 rather than +1/-1
    train_labels = np.r_[np.ones((n_pos, )), np.zeros((n_neg, ))]

    train_data_iter = mx.io.NDArrayIter(
        data=train_data, 
        batch_size=batch_size,
        label=train_labels,
        shuffle=True)
    
    #####################################
    
    test_features_pos = test_features[structure]
    n_pos = len(test_features_pos)
    
    test_features_neg = np.concatenate([test_features[neg_class] for neg_class in neg_classes \
                                   if neg_class in test_features])
    n_neg = len(test_features_neg)
    
    test_data = np.concatenate([test_features_pos, test_features_neg])
    # For cnn, labels must be 0/1 rather than +1/-1
    test_labels = np.r_[np.ones((n_pos, )), np.zeros((n_neg, ))]

    test_data_iter = mx.io.NDArrayIter(
        data=test_data, 
        batch_size=batch_size,
        label=test_labels,
        shuffle=True)
    
    t = time.time()
    prefix = os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, model_name + '_' + structure)
    mod_score = fit(new_sym, new_args, aux_params, train_data_iter, test_data_iter, 
                    batch_size, num_gpus, num_epoch=10, epoch_end_callback=my_epoch_end_callback(prefix, period=5))
#     clf.fit(train_data, train_labels, sample_weight=sample_weights)
    sys.stderr.write('Fitting classifier: %.2f seconds\n' % (time.time() - t))
    
#     clf_fp = DataManager.get_classifier_filepath(classifier_id=classifier_id, structure=structure)
#     create_parent_dir_if_not_exists(clf_fp)
#     joblib.dump(clf, clf_fp)
#     upload_to_s3(clf_fp)

LRt
rm -rf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_5N.hdf && mkdir -p /home/yuncong/CSHL_classifiers/datasets/dataset_60
aws s3 cp s3://mousebrainatlas-data/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_5N.hdf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_5N.hdf


Child returned 0
Child returned 1
0.73 seconds.
Cannot load dataset images for label LRt_surround_500_5N: [Errno 2] No such file or directory: '/home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_5N.hdf'
Child returned 0


rm -rf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_6N.hdf && mkdir -p /home/yuncong/CSHL_classifiers/datasets/dataset_60
aws s3 cp s3://mousebrainatlas-data/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_6N.hdf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_6N.hdf


Child returned 1
0.69 seconds.
Cannot load dataset images for label LRt_surround_500_6N: [Errno 2] No such file or directory: '/home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_6N.hdf'
Child returned 0


rm -rf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_7N.hdf && mkdir -p /home/yuncong/CSHL_classifiers/datasets/dataset_60
aws s3 cp s3://mousebrainatlas-data/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_7N.hdf /home/yuncong/CSHL_classifiers/datasets/dataset_60/patch_images_LRt_surround_500_7N.hdf


...

Child returned 0
Child returned 1
0.66 seconds.
Cannot load dataset images for label noclass: [Errno 2] No such file or directory: '/home/yuncong/CSHL_classifiers/datasets/dataset_62/patch_images_noclass.hdf'
INFO:root:Epoch[0] Batch [10]	Speed: 161.83 samples/sec	Train-accuracy=0.590909
INFO:root:Epoch[0] Batch [20]	Speed: 164.76 samples/sec	Train-accuracy=0.625000
INFO:root:Epoch[0] Batch [30]	Speed: 164.01 samples/sec	Train-accuracy=0.668750
INFO:root:Epoch[0] Batch [40]	Speed: 160.67 samples/sec	Train-accuracy=0.743750
INFO:root:Epoch[0] Batch [50]	Speed: 163.54 samples/sec	Train-accuracy=0.787500
INFO:root:Epoch[0] Batch [60]	Speed: 165.05 samples/sec	Train-accuracy=0.768750
INFO:root:Epoch[0] Batch [70]	Speed: 164.30 samples/sec	Train-accuracy=0.837500
INFO:root:Epoch[0] Batch [80]	Speed: 160.65 samples/sec	Train-accuracy=0.737500
INFO:root:Epoch[0] Batch [90]	Speed: 164.73 samples/sec	Train-accuracy=0.768750
INFO:root:Epoch[0] Batch [100]	Speed: 164.33 samples/sec	Train-accuracy

aws s3 cp /home/yuncong/mxnet_models/inception-bn-blue-softmax/inception-bn-blue-softmax_LRt-symbol.json s3://mousebrainatlas-data/mxnet_models/inception-bn-blue-softmax/inception-bn-blue-softmax_LRt-symbol.json


Child returned 0
0.82 seconds.


aws s3 cp /home/yuncong/mxnet_models/inception-bn-blue-softmax/inception-bn-blue-softmax_LRt-0000.params s3://mousebrainatlas-data/mxnet_models/inception-bn-blue-softmax/inception-bn-blue-softmax_LRt-0000.params


Child returned 0
1.54 seconds.
INFO:root:Epoch[0] Validation-accuracy=0.735000
INFO:root:Epoch[1] Batch [10]	Speed: 167.35 samples/sec	Train-accuracy=0.846591
INFO:root:Epoch[1] Batch [20]	Speed: 162.22 samples/sec	Train-accuracy=0.800000
INFO:root:Epoch[1] Batch [30]	Speed: 164.51 samples/sec	Train-accuracy=0.850000
INFO:root:Epoch[1] Batch [40]	Speed: 164.71 samples/sec	Train-accuracy=0.837500
INFO:root:Epoch[1] Batch [50]	Speed: 160.41 samples/sec	Train-accuracy=0.825000
INFO:root:Epoch[1] Batch [60]	Speed: 160.77 samples/sec	Train-accuracy=0.843750
INFO:root:Epoch[1] Batch [70]	Speed: 145.83 samples/sec	Train-accuracy=0.862500
INFO:root:Epoch[1] Batch [80]	Speed: 135.60 samples/sec	Train-accuracy=0.806250
INFO:root:Epoch[1] Batch [90]	Speed: 138.69 samples/sec	Train-accuracy=0.850000
INFO:root:Epoch[1] Batch [100]	Speed: 155.66 samples/sec	Train-accuracy=0.875000
INFO:root:Epoch[1] Batch [110]	Speed: 158.32 samples/sec	Train-accuracy=0.825000
INFO:root:Epoch[1] Batch [120]	Speed: 1